In [36]:
import sys
sys.path.append("/home/j-gunmo/desktop/00.my-project/17.P-Stage-T1003/4-STAGE/")

from fe.agg import AggFeBase
from fe.seq import SeqFeBase

from fe.agg import (
    MakeCorrectCount, 
    MakeCorrectPercent, 
    MakeQuestionCount, 
    MakeTopNCorrectPercent
)

from fe.seq import (
    SplitAssessmentItemID,
    MakeFirstClass,
    MakeSecondClass,
    MakeTimeDiff,
    MakeYMD,
    ConvertTime
)

import easydict
from fe.feature import FEPipeline
from dkt_dataset import Preprocess
from utils import get_args, get_root_dir
from IPython.display import clear_output

In [101]:
def make_df_pipeline(train_data, test_data):
    train_data = MakeFirstClass.transform(train_data, key="train")
    test_data = MakeFirstClass.transform(test_data, key="test")
    
    feature_data = pd.merge(train_data, train_data.groupby(["userID", "firstClass"]).answerCode.mean(), 
                        on=["userID", "firstClass"], how="inner", suffixes=['', '_userper'])
    
    t_data = pd.merge(test_data, test_data.groupby(["userID", "firstClass"]).answerCode.mean(), 
                        on=["userID", "firstClass"], how="inner", suffixes=['', '_userper'])
    
    feature_data = pd.merge(feature_data, 
                        train_data.groupby("assessmentItemID").answerCode.mean(),
                       on="assessmentItemID", how="inner", suffixes=['', '_assper'])
    
    t_data = pd.merge(t_data, 
                        train_data.groupby("assessmentItemID").answerCode.mean(),
                       on="assessmentItemID", how="inner", suffixes=['', '_assper'])
    
    feature_data['firstClass'] = feature_data['testId'].apply(lambda x: int(x[2:3]))
    t_data['firstClass'] = t_data['testId'].apply(lambda x: int(x[2:3]))
    
    feature_data = pd.merge(feature_data, 
                        feature_data.groupby("firstClass").answerCode.mean(),
                       on="firstClass", how="inner", suffixes=["", "_firper"])
    
    t_data = pd.merge(t_data, 
                        feature_data.groupby("firstClass").answerCode.mean(),
                       on="firstClass", how="inner", suffixes=["", "_firper"])
    
    feature_data = pd.merge(feature_data, 
                        feature_data.groupby("testId").answerCode.mean(),
                       on="testId", how="inner", suffixes=["", "_testper"])
    
    t_data = pd.merge(t_data, 
                        feature_data.groupby("testId").answerCode.mean(),
                       on="testId", how="inner", suffixes=["", "_testper"])
    
    feature_data = pd.merge(feature_data, 
                        feature_data.groupby(["userID", "KnowledgeTag"]).answerCode.mean(),
                       on=["userID", "KnowledgeTag"], how="inner", suffixes=["", "_knowper"])
    
    t_data = pd.merge(t_data, 
                        feature_data.groupby(["userID", "KnowledgeTag"]).answerCode.mean(),
                       on=["userID", "KnowledgeTag"], how="inner", suffixes=["", "_knowper"])
    
    
    feature_data = feature_data.rename(columns={
        "answerCode_firper" : "firPer",
        "answerCode_userper": "userPer",
        "answerCode_assper": "assPer",
        "answerCode_testper" : "testPer",
        "answerCode_knowper" : "secPer"
    })

    t_data = t_data.rename(columns={
        "answerCode_firper" : "firPer",
        "answerCode_userper": "userPer",
        "answerCode_assper": "assPer",
        "answerCode_testper" : "testPer",
        "answerCode_knowper" : "secPer"
    })

    return feature_data, t_data

In [102]:
args = get_args()
args.data_dir = "../../input/data/train_dataset/"
args.root_dir = get_root_dir("../last_lstm/")

In [103]:
import os.path as p
import pandas as pd

train_data = pd.read_csv(p.join(args.data_dir, "train_data.csv"))
test_data = pd.read_csv(p.join(args.data_dir, "test_data.csv"))

In [104]:
import torch
from trainer import DKTTrainer
from models.lstm.model import LSTM

class FeatureTestTrainer(DKTTrainer):
    def _process_batch(self, batch):
        batch['mask'] = batch['mask'].type(torch.FloatTensor)
        batch["answerCode"] = batch["answerCode"].type(torch.FloatTensor)

        batch["interaction"] = batch["answerCode"] + 1
        batch["interaction"] = batch["interaction"].roll(shifts=1, dims=1)
        batch["mask"] = batch["mask"].roll(shifts=1, dims=1)
        batch["mask"][:, 0] = 0
        batch["interaction"] = (batch["interaction"] * batch["mask"]).to(torch.int64)
        
        
        for k in self.args.n_linears: # 수치형
            batch[k] = batch[k].type(torch.FloatTensor)
            
        for k, v in self.args.n_embeddings.items(): # 범주형
            batch[k] = batch[k].to(torch.int64)
            
        for k in batch.keys():
            batch[k] = batch[k].to(self.args.device)
        
        return batch

In [105]:
train_data["org_index"] = train_data.index
test_data["org_index"] = test_data.index

In [106]:
train_set, test_set = make_df_pipeline(train_data, test_data)

load features /home/j-gunmo/features/train_make_first_class.pkl to dataframe ... 
load features /home/j-gunmo/features/test_make_first_class.pkl to dataframe ... 


In [107]:
train_set = train_set.sort_values(["org_index"], axis=0).reset_index(drop=True)
test_set = test_set.sort_values(["org_index"], axis=0).reset_index(drop=True)

In [108]:
test_set.head(2)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,org_index,firstClass,userPer,assPer,firPer,testPer,secPer
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,0,5,0.653132,0.646789,0.658208,0.560944,0.641379
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,1,5,0.653132,0.628440,0.658208,0.560944,0.641379


In [109]:
fe_list = [
    SplitAssessmentItemID,
    ConvertTime,
    MakeFirstClass,
    MakeSecondClass,
    MakeCorrectCount,
    MakeQuestionCount,
    MakeCorrectPercent
]

fe_pipeline = FEPipeline(
    args, fe_list
)

columns = [
    "userID",
    "answerCode",
    "testPaper",
    "firstClass",
    "secondClass",
    "timeSec"
]

add_columns =[
    'userPer',
    'assPer',
    'firPer',
    'testPer',
    'secPer'
]

pre_encodes = {
    "label" : ['testPaper', 'firstClass', 'secondClass'],
    "min_max" : ['userPer', 'assPer', 'firPer', 'testPer', 'secPer'],
    "std" : ["timeSec"]
}

preprocess = Preprocess(args, fe_pipeline, columns)

In [110]:
preprocess.feature_engineering()

Feature Engineering Start ... 
Feature Engineering Start ... 
Feature Engineering Start ... 
Feature Engineering Start ... 
Feature Engineering Start ... 


load features /home/j-gunmo/features/train_split_assessmentitem_id.pkl to dataframe ... 



Feature Engineering Name: split_assessmentitem_id

Feature Engineering Name: split_assessmentitem_id

Feature Engineering Name: split_assessmentitem_id

Feature Engineering Name: split_assessmentitem_id

Feature Engineering Name: split_assessmentitem_id



testPaper       : 시험지 번호입니다.

testPaper       : 시험지 번호입니다.

testPaper       : 시험지 번호입니다.

testPaper       : 시험지 번호입니다.

testPaper       : 시험지 번호입니다.


dtype: object
dtype: object
dtype: object
dtype: object
dtype: object


...

INDEX 8000: 0.6911196911196911
INDEX 8000: 0.6911196911196911
INDEX 8000: 0.6911196911196911
INDEX 8000: 0.6911196911196911
INDEX 8000: 0.6911196911196911


INDEX 9000: 0.6911196911196911
INDEX 9000: 0.6911196911196911
INDEX 9000: 0.6911196911196911
INDEX 9000: 0.6911196911196911
INDEX 9000: 0.6911196911196911


Feature Engineering End ... 
Feature Engineering End ... 
Feature Engineering End ... 
Feature Engineering End ... 
Feature Engineering End ... 


Original DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag'],
      dtype='object')
Original DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag'],
      dtype='object')
Original DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag'],
      dtype='object')
Original DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag'],
      dtype='object')
Original DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag'],
      dtype='object')


Feature Added DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'testPaper', 'testPaperCnt', 'timeSec', 'firstClass',
       'secondClass', 'correctCnt', 'quesCnt', 'correctPer'],
      dtype='object')
Feature Added DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'testPaper', 'testPaperCnt', 'timeSec', 'firstClass',
       'secondClass', 'correctCnt', 'quesCnt', 'correctPer'],
      dtype='object')
Feature Added DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'testPaper', 'testPaperCnt', 'timeSec', 'firstClass',
       'secondClass', 'correctCnt', 'quesCnt', 'correctPer'],
      dtype='object')
Feature Added DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'testPaper', 'testPaperCnt', 'timeSec', 'firstClass',
       '

In [111]:
preprocess.datas['train'] = pd.concat([preprocess.datas['train'], train_set[add_columns]], axis=1)
preprocess.datas['test'] = pd.concat([preprocess.datas['test'], test_set[add_columns]], axis=1)

In [112]:
preprocess.columns += add_columns

In [113]:
print(preprocess.columns)

['userID', 'answerCode', 'testPaper', 'firstClass', 'secondClass', 'timeSec', 'userPer', 'assPer', 'firPer', 'testPer', 'secPer']


In [114]:
preprocess.split_data()

Split based on User
Split based on User
Split based on User
Split based on User
Split based on User


Original Train Dataset: 2266586
Original Train Dataset: 2266586
Original Train Dataset: 2266586
Original Train Dataset: 2266586
Original Train Dataset: 2266586


Split Train Dataset: 1582219
Split Train Dataset: 1582219
Split Train Dataset: 1582219
Split Train Dataset: 1582219
Split Train Dataset: 1582219


Split Valid Dataset: 684367
Split Valid Dataset: 684367
Split Valid Dataset: 684367
Split Valid Dataset: 684367
Split Valid Dataset: 684367


In [118]:
preprocess.datas['train'].head(3)

,userID,answerCode,testPaper,firstClass,secondClass,timeSec,userPer,assPer,firPer,testPer,secPer
0,0,1,975,5,618,-1.435761,0.791908,0.984954,0.738852,0.987599,0.971153
1,0,1,975,5,619,-1.435760,0.791908,0.966010,0.738852,0.987599,0.918119
2,0,1,975,5,619,-1.435759,0.791908,0.909180,0.738852,0.987599,0.918119


In [116]:
preprocess.scaling(pre_encodes)

Preprocessing Labels .. 
Preprocessing Labels .. 
Preprocessing Labels .. 
Preprocessing Labels .. 
Preprocessing Labels .. 


Label Columns: ['testPaper', 'firstClass', 'secondClass']
Label Columns: ['testPaper', 'firstClass', 'secondClass']
Label Columns: ['testPaper', 'firstClass', 'secondClass']
Label Columns: ['testPaper', 'firstClass', 'secondClass']
Label Columns: ['testPaper', 'firstClass', 'secondClass']



Length of testPaper            : 1538

Length of testPaper            : 1538

Length of testPaper            : 1538

Length of testPaper            : 1538

Length of testPaper            : 1538


Before : 0    060001
1    060001
2    060001
3    060001
4    060001
5    060001
6    060003
7    060003
8    060003
9    060003
Name: testPaper, dtype: object
Before : 0    060001
1    060001
2    060001
3    060001
4    060001
5    060001
6    060003
7    060003
8    060003
9    060003
Name: testPaper, dtype: object
Before : 0    060001
1    060001
2    060001
3    060001
4    060001
5    060001
6    060003
7    060003
8    060003
9    060003
Name: testPaper, dtype: object
Before : 0    060001
1    060001
2    060001
3    060001
4    060001
5    060001
6    060003
7    060003
8    060003
9    060003
Name: testPaper, dtype: object
Before : 0    060001
1    060001
2    060001
3    060001
4    060001
5    060001
6    060003
7    060003
8    060003
9    060003
Name: testPaper, dtype: object


After : 0    975
1    975
2    975
3    975
4    975
5    975
6    977
7    977
8    977
9    977
Name: testPaper, dtype: int64
After : 0    975
1    975
2    975
3    975
4    975
5    975
6    977
7    977
8    977
9    977
Name: testPaper, dtype: int64
After : 0    975
1    975
2    975
3    975
4    975
5    975
6    977
7    977
8    977
9    977
Name: testPaper, dtype: int64
After : 0    975
1    975
2    975
3    975
4    975
5    975
6    977
7    977
8    977
9    977
Name: testPaper, dtype: int64
After : 0    975
1    975
2    975
3    975
4    975
5    975
6    977
7    977
8    977
9    977
Name: testPaper, dtype: int64


...

Min Max Columns: ['userPer', 'assPer', 'firPer', 'testPer', 'secPer']
Min Max Columns: ['userPer', 'assPer', 'firPer', 'testPer', 'secPer']
Min Max Columns: ['userPer', 'assPer', 'firPer', 'testPer', 'secPer']
Min Max Columns: ['userPer', 'assPer', 'firPer', 'testPer', 'secPer']
Min Max Columns: ['userPer', 'assPer', 'firPer', 'testPer', 'secPer']


MAX: [1.         0.99630996 0.80087621 0.95547445 0.97777778] MIN: [0.         0.04942966 0.44994765 0.32718579 0.18894009]
MAX: [1.         0.99630996 0.80087621 0.95547445 0.97777778] MIN: [0.         0.04942966 0.44994765 0.32718579 0.18894009]
MAX: [1.         0.99630996 0.80087621 0.95547445 0.97777778] MIN: [0.         0.04942966 0.44994765 0.32718579 0.18894009]
MAX: [1.         0.99630996 0.80087621 0.95547445 0.97777778] MIN: [0.         0.04942966 0.44994765 0.32718579 0.18894009]
MAX: [1.         0.99630996 0.80087621 0.95547445 0.97777778] MIN: [0.         0.04942966 0.44994765 0.32718579 0.18894009]


Preprocessing Min Max .. 
Preprocessing Min Max .. 
Preprocessing Min Max .. 
Preprocessing Min Max .. 
Preprocessing Min Max .. 


Standard Columns: ['timeSec']
Standard Columns: ['timeSec']
Standard Columns: ['timeSec']
Standard Columns: ['timeSec']
Standard Columns: ['timeSec']


MEAN: [1.59503981e+09] VAR: [4.91254683e+13]
MEAN: [1.59503981e+09] VAR: [4.91254683e+13]
MEAN: [1.59503981e+09] VAR: [4.91254683e+13]
MEAN: [1.59503981e+09] VAR: [4.91254683e+13]
MEAN: [1.59503981e+09] VAR: [4.91254683e+13]


In [117]:
preprocess.data_augmentation(choices=[1, 3])

Use the test datast for data augmentation
Use the test datast for data augmentation
Use the test datast for data augmentation
Use the test datast for data augmentation
Use the test datast for data augmentation


Before Length: 1582219
Before Length: 1582219
Before Length: 1582219
Before Length: 1582219
Before Length: 1582219


After Length: 1841589
After Length: 1841589
After Length: 1841589
After Length: 1841589
After Length: 1841589


Group By (userID, firstClass)
Group By (userID, firstClass)
Group By (userID, firstClass)
Group By (userID, firstClass)
Group By (userID, firstClass)


Group By (userID, firstClass) Length: 14429
Group By (userID, firstClass) Length: 14429
Group By (userID, firstClass) Length: 14429
Group By (userID, firstClass) Length: 14429
Group By (userID, firstClass) Length: 14429


Group By (userID, firstClass) Length: 5310
Group By (userID, firstClass) Length: 5310
Group By (userID, firstClass) Length: 5310
Group By (userID, firstClass) Length: 5310
Group By (userID, firstClass) Length: 5310


Group By (userID, firstClass) Length: 1987
Group By (userID, firstClass) Length: 1987
Group By (userID, firstClass) Length: 1987
Group By (userID, firstClass) Length: 1987
Group By (userID, firstClass) Length: 1987


In [120]:
args.columns = columns[1:]

In [121]:
train_dataset = preprocess.datas['train_grouped']
valid_dataset = preprocess.datas['valid_grouped']
test_dataset = preprocess.datas['test_grouped']

In [122]:
args.hidden_dim = 1024
args.batch_size = 256

In [123]:
trainer = FeatureTestTrainer(args, LSTM)

In [124]:
trainer.debug(train_dataset, valid_dataset, test_dataset)

In [78]:
trainer.run_cv(train_dataset, valid_dataset, test_dataset, folds=5, seeds=[0, 1, 2, 3, 4])

wandb: Currently logged in as: ggm1207 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Training steps: 0 Loss: 0.6900123357772827
Training steps: 50 Loss: 0.5907658338546753
Training steps: 100 Loss: 0.6147000789642334
VALID AUC : 0.7775235145970394 ACC : 0.709726443768997



Training steps: 0 Loss: 0.6053119897842407
Training steps: 50 Loss: 0.5378944873809814
Training steps: 100 Loss: 0.6572806239128113
VALID AUC : 0.7812949732730264 ACC : 0.7092198581560284



Training steps: 0 Loss: 0.5924673676490784
Training steps: 50 Loss: 0.5560609698295593
Training steps: 100 Loss: 0.5315614938735962
VALID AUC : 0.7831839792351974 ACC : 0.7201114488348531



Training steps: 0 Loss: 0.48299431800842285
Training steps: 50 Loss: 0.5084896087646484
Training steps: 100 Loss: 0.6187272071838379
VALID AUC : 0.783579615542763 ACC : 0.7160587639311043



...

Training steps: 0 Loss: 0.6147621870040894
Training steps: 50 Loss: 0.5941517353057861
Training steps: 100 Loss: 0.48723554611206055
VALID AUC : 0.778465543807449 ACC : 0.7120060790273556



Training steps: 0 Loss: 0.5422154664993286
Training steps: 50 Loss: 0.5661067962646484
Training steps: 100 Loss: 0.6025415658950806
VALID AUC : 0.7786846176295519 ACC : 0.711499493414387



Training steps: 0 Loss: 0.5817445516586304
Training steps: 50 Loss: 0.5666242837905884
Training steps: 100 Loss: 0.6370506882667542
VALID AUC : 0.7784343575515786 ACC : 0.7092198581560284



Training steps: 0 Loss: 0.5707943439483643
Training steps: 50 Loss: 0.5356687903404236
Training steps: 100 Loss: 0.5926520824432373
VALID AUC : 0.7784059947427415 ACC : 0.7112462006079028



(0.7837383064719081, 0.7168186423505573)

In [126]:
auc, acc = trainer.run_cv(train_dataset, valid_dataset, test_dataset, folds=5, seeds=[0, 1, 2, 3, 4])
clear_output()
print(f"auc : {auc} acc : {acc}")
print(f"logging path: {trainer.prefix_save_path}")

auc : 0.8026846231477597 acc : 0.7294326241134752
logging path: ../last_lstm/LOG_[06.14_13:48]


In [178]:
import torch.nn as nn
from models.lstm.model import EmbeddingLayer, LinearLayer

class HiddenLayer(nn.Module):
    def __init__(self, args):
        super(HiddenLayer, self).__init__()
        
        self.args = args
        self.hidden_dim = self.args.hidden_dim
        self.hidden_layer = nn.Linear(2, self.hidden_dim * args.n_layers)
        
    def forward(self, batch):
        batch_hid = torch.cat([batch['userPer'][:, -1:], 
                               batch['testPer'][:, -1:]], 1)
        output = self.hidden_layer(batch_hid)
        output = output.reshape(-1, self.args.n_layers, self.hidden_dim).permute(1, 0, 2)
        return output.contiguous(), torch.empty_like(output).copy_(output).contiguous()
        

class LSTM(nn.Module):
    def __init__(self, args):
        super(LSTM, self).__init__()
        self.args = args
        self.device = args.device

        self.hidden_dim = self.args.hidden_dim
        self.n_layers = self.args.n_layers
        
        self.hid_layer = HiddenLayer(args)
        self.emb_layer = EmbeddingLayer(args, self.hidden_dim // 2)
        args.n_linears = ['secPer', 'timeSec', 'assPer']
        self.nli_layer = LinearLayer(args, self.hidden_dim // 2)
        
        self.comb_proj = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.lstm = nn.LSTM(self.hidden_dim, self.hidden_dim, self.n_layers, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(self.hidden_dim, 1)
        self.activation = nn.Sigmoid()

    def forward(self, batch):
        batch_size = batch["interaction"].size(0)

        embed = self.emb_layer(batch)
        nnbed = self.nli_layer(batch)

        embed = torch.cat([embed, nnbed], 2)
        X = self.comb_proj(embed)
        
        hidden = self.hid_layer(batch)
        out, hidden = self.lstm(X, hidden)
        out = out.contiguous().view(batch_size, -1, self.hidden_dim)

        out = self.fc(out)
        preds = self.activation(out).view(batch_size, -1)

        return preds

In [176]:
args.n_linears

['userPer', 'assPer', 'firPer', 'testPer', 'secPer', 'timeSec']

In [173]:
trainer = FeatureTestTrainer(args, LSTM)

In [175]:
auc, acc = trainer.run_cv(train_dataset, valid_dataset, test_dataset, folds=5, seeds=[0, 1, 2, 3, 4])
clear_output()
print(f"auc : {auc} acc : {acc}")
print(f"logging path: {trainer.prefix_save_path}")

auc : 0.8022710933055196 acc : 0.7281661600810537
logging path: ../last_lstm/LOG_[06.14_14:32]


In [177]:
args.n_linears = ['secPer', 'timeSec', 'assPer']

['userPer', 'assPer', 'firPer', 'testPer', 'secPer', 'timeSec']

In [179]:
trainer = FeatureTestTrainer(args, LSTM)

In [182]:
args.n_layers, args.hidden_dim

(2, 1024)

In [183]:
auc, acc = trainer.run_cv(train_dataset, valid_dataset, test_dataset, folds=5, seeds=[0, 1, 2, 3, 4])
clear_output()
print(f"auc : {auc} acc : {acc}")
print(f"logging path: {trainer.prefix_save_path}")

auc : 0.782386306241773 acc : 0.7152988855116514
logging path: ../last_lstm/LOG_[06.14_14:50]


In [188]:
from transformers import BertConfig, BertModel

In [189]:
class Bert(nn.Module):
    def __init__(self, args):
        super(Bert, self).__init__()
        self.args = args
        self.device = args.device

        # Defining some parameters
        self.hidden_dim = self.args.hidden_dim
        self.n_layers = self.args.n_layers
        
        self.emb_layer = EmbeddingLayer(args, self.hidden_dim // 2)
        self.nli_layer = LinearLayer(args, self.hidden_dim // 2)
        
        self.comb_proj = nn.Linear(self.hidden_dim, self.hidden_dim)

        # Bert config
        self.config = BertConfig(
            3,  # not used
            hidden_size=self.hidden_dim,
            num_hidden_layers=self.args.n_layers,
            num_attention_heads=self.args.n_heads,
            max_position_embeddings=self.args.max_seq_len,
        )

        # Defining the layers
        # Bert Layer
        self.encoder = BertModel(self.config)

        # Fully connected layer
        self.fc = nn.Linear(self.args.hidden_dim, 1)
        self.activation = nn.Sigmoid()

    def forward(self, batch):
        batch_size = batch["interaction"].size(0)
    
        embed = self.emb_layer(batch)
        nnbed = self.nli_layer(batch)
        
        embed = torch.cat([embed, nnbed], 2)
        X = self.comb_proj(embed)

        # Bert
        encoded_layers = self.encoder(inputs_embeds=X, attention_mask=batch["mask"])
        out = encoded_layers[0]
        out = out.contiguous().view(batch_size, -1, self.hidden_dim)
        out = self.fc(out)
        preds = self.activation(out).view(batch_size, -1)

        return preds

In [193]:
args.n_heads = 4

In [196]:
trainer = FeatureTestTrainer(args, Bert)

In [ ]:
auc, acc = trainer.run_cv(train_dataset, valid_dataset, test_dataset, folds=5, seeds=[0, 1, 2, 3, 4])
clear_output()
print(f"auc : {auc} acc : {acc}")
print(f"logging path: {trainer.prefix_save_path}")

epoch,14
train_loss,0.56348
train_auc,0.7806
train_acc,0.71072
valid_auc,0.7776
valid_acc,0.71327
_runtime,100
_timestamp,1623650488
_step,14


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▃▂▂▂▁▂▁▁▁▁▁▁▁▁
train_auc,▁▆▆▇▇▇▇█████▇██
train_acc,▁▆▆▆▇▇▇▇█▇█████
valid_auc,▁▄▆▆▇▇▆▇▇██▇█▇█
valid_acc,▃▁▅▆▇█▆▆▁▅██▅▇▆
_runtime,▁▂▂▃▃▄▄▅▅▆▆▇▇██
_timestamp,▁▂▂▃▃▄▄▅▅▆▆▇▇██
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█


wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Training steps: 0 Loss: 0.736411452293396
Training steps: 50 Loss: 0.6395953893661499
VALID AUC : 0.8274023437500001 ACC : 0.7454407294832827



Training steps: 0 Loss: 0.4838271141052246
Training steps: 50 Loss: 0.5337360501289368
VALID AUC : 0.8348422080592106 ACC : 0.7462006079027356



...

Training steps: 0 Loss: 0.48088812828063965
Training steps: 50 Loss: 0.5347450375556946
VALID AUC : 0.8344077410744076 ACC : 0.747467071935157



Training steps: 0 Loss: 0.4528772234916687
Training steps: 50 Loss: 0.5596789717674255
VALID AUC : 0.8324837658170992 ACC : 0.7411347517730497



Training steps: 0 Loss: 0.5551417469978333
Training steps: 50 Loss: 0.48741453886032104
VALID AUC : 0.8327412027412027 ACC : 0.754305977710233



Training steps: 0 Loss: 0.5082675814628601
Training steps: 50 Loss: 0.5236749649047852
VALID AUC : 0.832568209234876 ACC : 0.7459473150962512



Training steps: 0 Loss: 0.45531702041625977


In [198]:
auc, acc

(0.8328989350284008, 0.747872340425532)